<a href="https://colab.research.google.com/github/sean-otoole/HODD/blob/main/PDTx_pipeline/PDTx_04_Perturbation_DA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#fixes a compatibility issue with the PrecollatorForGeneAndCellClassification class

!pip install --upgrade transformers==4.41
!pip install peft==0.10.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 92.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [2]:
#first mount the drive

from google.colab import drive
import os

# Mount Google Drive to access files
drive.mount('/content/drive')

# Change the working directory to the project folder in Google Drive
os.chdir("/content/drive/MyDrive/HODD/")

# Install Git Large File Storage (LFS) for handling large files in Git repositories
# !git lfs install

#Clone the Geneformer repository (commented out to avoid repeated cloning)
# !git clone https://huggingface.co/ctheodoris/Geneformer

# Navigate to the Geneformer directory
%cd Geneformer

# Install Geneformer package locally
# I found that installing Genformer first helped with a lot of the version conflict issues
!pip install .

# Install required libraries without outputting installation logs
!pip install anndata scanpy tdigest datasets

# Import necessary modules and libraries
import numpy
import transformers
import sklearn
import pickle
import sklearn

Mounted at /content/drive
/content/drive/MyDrive/HODD/Geneformer
Processing /content/drive/MyDrive/HODD/Geneformer
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 108.5 MB/s et

In [1]:
from geneformer import InSilicoPerturber
from geneformer import InSilicoPerturberStats
from geneformer import EmbExtractor

In [15]:
cell_states_to_model={"state_key": "disease_state",
                      "start_state": "Parkinson_disease",
                      "goal_state": "normal"}

In [16]:
embex = EmbExtractor(model_type="CellClassifier", # if using previously fine-tuned cell classifier model
                     num_classes=2,
                     max_ncells=2000,
                     emb_layer=0,
                     summary_stat="exact_mean",
                     forward_batch_size=100,
                     nproc=4)

In [ ]:
fine_tuned_model = '/content/drive/MyDrive/HODD/models/geneformer_finetuned_v1/250414_geneformer_cellClassifier_park_classifier/ksplit1/'
output_dir = '/content/drive/MyDrive/HODD/perturbation_analysis_overexpress_output'
current_prefix = 'park_overexpress'
input_data =  '/content/drive/MyDrive/HODD/Geneformer_parkinsons_files/dopamine_subset_test_tokenized.dataset/'   #would like to make this all of the data

state_embs_dict = embex.get_state_embs(cell_states_to_model,
                                       fine_tuned_model,
                                       input_data,
                                       output_dir,
                                       current_prefix)

In [19]:
#create a dictionary for adding ensembl_gene_id which the nomenclature used by geneformer
#may need to be executed multiple times due to issues with biomart

# !pip install pybiomart

# import scanpy as sc
# import pybiomart

# annot = sc.queries.biomart_annotations(
#     "hsapiens",
#     ["ensembl_gene_id", "hgnc_symbol"],
# )

# gene_mapping = annot.set_index("hgnc_symbol")["ensembl_gene_id"].to_dict()

In [20]:
# genes_to_perturb = [
#     # Parkinson’s disease-associated
#     "SNCA", "LRRK2", "PARK7", "PINK1", "PRKN",
#     "VPS35", "ATP13A2", "MAPT", "UCHL1",
#     "TMEM175", "SYNJ1",

#     # Control genes (highly expressed)
#     "ACTB", "GAPDH", "RPL13A", "B2M", "TUBA1B"
# ]

# genes_to_perturb = ["SNCA"]

# genes_to_perturb_ensembl = [gene_mapping.get(gene, gene) for gene in genes_to_perturb]

# print(genes_to_perturb_ensembl)

In [21]:
isp = InSilicoPerturber(perturb_type="overexpress",
                        perturb_rank_shift=None,
                        genes_to_perturb="all",
                        # genes_to_perturb=genes_to_perturb_ensembl,
                        combos=0,
                        anchor_gene=None,
                        model_type="CellClassifier", # if using previously fine-tuned cell classifier model
                        num_classes=2,
                        emb_mode="cls",   #should be cell - I think, will need to check
                        cell_emb_style="mean_pool",
                        cell_states_to_model=cell_states_to_model,
                        state_embs_dict=state_embs_dict,
                        max_ncells=400,
                        emb_layer=0,
                        forward_batch_size=200,
                        nproc=1)

In [ ]:
# outputs intermediate files from in silico perturbation
isp.perturb_data(fine_tuned_model,
                 input_data,
                 output_dir,
                 "parkinson_overexpress_stats")

In [ ]:
ispstats = InSilicoPerturberStats(mode="goal_state_shift",
                                  genes_perturbed="all",
                                  # genes_perturbed = genes_to_perturb_ensembl,
                                  combos=0,
                                  anchor_gene=None,
                                  cell_states_to_model=cell_states_to_model)

In [ ]:
# extracts data from intermediate files and processes stats to output in final .csv
ispstats.get_stats(output_dir, # this should be the directory
                   None,
                   output_dir,
                   "parkinson_delete_stats")